In [5]:
import serial
import numpy as np
np.set_printoptions(precision=3)

In [6]:
def flip(x, i, j):
    return x ^ ((1 << i) | (1 << j))

def get_spin(x, i):
    return (x >> i) & 1

def shift(x, N, k = 1): # k = number of steps to shift by
    return ((x << k) | (x >> (N - k))) & ((1 << N) - 1)


def findstate(basis, state):
    # can be implemented explicitely using binary search for example
    return basis.index(state) if state in basis else -1

In [8]:
def checkstate_k(s, N, k):
    t = s

    for i in range(1, N+1):
        t = shift(t, N)
        if t < s: return -1 # if a smaller state is found, s is not the representative, return -1
        elif t == s: # after i shifts, back to original state
            if k % (N // i) != 0:  return -1 # momentum k INcompatible with periodicity i (R = i the periodicity)
            else: return i # R = i, compatible with periodicity i
    
    return -1 # periodicity not compatible with k

def representative_k(s, N, k):
    # find the representative of the state s, which is the smallest state that can be obtained by translating
    r = s
    t = s
    l = 0
    for i in range(1, N+1):
        t = shift(t, N)
        if t < r: 
            r = t
            l = i
    
    return r, l # representative, number of translations

### Implementation of FPGA functions

In [ ]:
def fpga(ser, N, x):
    # send state x and receive answer on uart to fpga
    
    # prepare message
    msg = x.to_bytes((N + 7) // 8, byteorder='big') # convert to bytes

    # send message
    ser.reset_input_buffer()  # Clear the input buffer
    ser.reset_output_buffer()  # Clear the output buffer
    ser.write(msg)

    # receive data
    l = int.from_bytes(ser.readline(), byteorder='big')
    r = shift(x, N, l)
    print(l, bin(r))

    return r, l

def chekstate_fpga(ser, x, N, k):
    r, l = fpga(ser, N, x)

    if r != x: # means that not representative
        return -1
    else:
        # if r = x it means that the state is a valid representative, 
        # and in this case the number of steps to reach it is simply the periodicity
        
        if k % (N // l) != 0:  return -1 # period not compatible with k
        else: return l

def representative_fpga(ser, x, N, k):
    r, l = fpga(ser, N, x)
    
    # here if r = x and x is the rep, then we get the periodicty and not l = 0
    # this allows the fpga output to also be interpreted for checkstate (see comment there)
    
    if r == x: l = 0 
    
    return r, l

### Testing FPGA functions (comparing with non FPGA)

In [9]:
ser = serial.Serial('COM4', 9600, timeout=1)

N = 16
k = 0

for x in range(1 << N):
    r, l = representative_k(x, N, k)
    R = checkstate_k(x, N, k)

    r_fpga, l_fpga = representative_fpga(ser, x, N, k)
    R_fpga = chekstate_fpga(ser, x, N, k)

    assert r == r_fpga, f"Mismatch in representative: x={bin(x)}, cpu={bin(r)}, fpga={bin(r_fpga)}"
    assert l == l_fpga, f"Mismatch in number of translations: x={bin(x)}, cpu={l}, fpga={l_fpga}"
    assert R == R_fpga, f"Mismatch in state check: x={bin(x)}, cpu={R}, fpga={R_fpga}"

SerialException: [Errno 2] could not open port COM4: [Errno 2] No such file or directory: 'COM4'